## Simplifying State-wise Electricity Tariff

This simplification is being done for the electric cooking analysis.

Following are the assumptions:
1) BoP Consumer prices are for bottom of the pyramid / marginalized consumers.
2) Each state is assumed to have one distribution utility. If there are multiple utilities within the same state, the slab-wise average is assumed to be the retail tariff price for the state.
3) In order to maintain uniform slabs across the states and the country, each slab assumed is of 100 units. For e.g., first slab is 0-100 units, second slab is 101-200 units.
4) For states having slabs less than 100 units, weighted average is considered for calculating the average slab-wise retail price.
5) LIG - BPL / 0 - 50 units, MIG -  50 to 200 units, HIG - 200 - 400 units

### 1. Importing Libraries

In [49]:
import pandas as pd
import numpy as np

### 2. Loading file

In [50]:
etr1 = pd.read_excel('electric-tariff.xlsx')
etr1.columns = etr1.iloc[1]
etr1.drop(index = [0,1], inplace = True)
etr1.reset_index(drop=True, inplace=True)
#etr2 = etr2.drop(columns='index')
#ethd1 = etr2.head()
#ethd1

In [ ]:
print(etr1)

### 3. Removing unwanted columns

In [51]:
etr1.drop(columns=['Name of DISCOMs', 'Area','Year','All Units (Higher Load/HT-Domestic)','Source'], inplace = True)#.reset_index()
#etr4 = etr3.rename({'BPL/KJ-BJ Consumers/Rural/Life Line Connection':'BoP'}, axis = 1) #renaming BPL tariff to BoP consumer
etr1.rename(columns={'Name of States':'State','BPL/KJ-BJ Consumers/Rural/Life Line Connection':'BoP'}, inplace=True)
etr1.replace('Nil', np.nan, inplace = True)

#ethd2 = etr4.head()
#ethd2

In [52]:
print(etr1['BoP'].dtype)
print(etr1['0-50'].dtype)

float64
float64


### 4. Grouping by States for state-wise average slab-wise retail 

In [53]:
#etr4 = etr4.replace(['Nil', np.nan]) # replacing Nil values of BoP with NaN
etr2 = etr1.groupby('State')[['BoP',
       '0-50', '51-100', '0-100', '0-120', '0-150', '0-200', '101-150',
       '101-250', '101-200', '121-240', '151-200', '151-250', '201-250',
       '151-300', '201-300', '151-400', '301-400', '301-500', '401-500',
       '201-500', '251-500', '401-600', '401-800', '601-900', '801-1200',
       '101 & Above', '201 & Above', '251 & Above', '301 & Above',
       '401 & Above', '501 & Above', '601 & Above', '801 & Above',
       '901 & Above', '1201 & Above']].mean().round(2)


In [ ]:
list(etr2.columns)

### 5. Recalibrating slabs to 100 units per slab

In [ ]:
etr2['BoP'].fillna(etr2['0-50'], inplace=True)
#df['A'].fillna(df['B'], inplace=True)


In [54]:
print(etr2)
etr2.to_excel('v2-el_price_rev.xlsx')

1                             BoP  0-50  51-100  0-100  0-120  0-150  0-200  \
State                                                                         
Andaman and Nicobar Islands  2.10   NaN     NaN   2.50    NaN    NaN    NaN   
Andhra Pradesh               1.45  2.65    3.35    NaN    NaN    NaN    NaN   
Arunachal Pradesh            2.65   NaN     NaN    NaN    NaN    NaN    NaN   
Assam                        4.25   NaN     NaN    NaN   4.90    NaN    NaN   
Bihar                        6.10  6.10    6.40    NaN    NaN    NaN    NaN   
Chandigarh                    NaN   NaN     NaN    NaN    NaN    2.5    NaN   
Chhattisgarh                  NaN   NaN     NaN   3.60    NaN    NaN    NaN   
Dadra and Nagar Haveli        NaN  1.40    2.10    NaN    NaN    NaN    NaN   
Daman and Diu                 NaN   NaN     NaN   1.40    NaN    NaN    NaN   
Delhi                         NaN   NaN     NaN   3.00    NaN    NaN    NaN   
Goa                           NaN   NaN     NaN   1.

In [ ]:
etr2 = etr2.assign(sl1 =etr2[['0-50', '51-100']].mean(axis=1))
etr2['0-100'].fillna(value=etr2['sl1'], inplace=True)
etr2.drop(columns=['0-50','51-100','sl1'],inplace=True)

etr2 = etr2.assign(sl2 =etr2[['0-100', '101-200']].mean(axis=1))
etr2['0-200'].fillna(value=etr2['sl2'], inplace=True)
etr2.drop(columns=['0-100', '101-200','sl2'],inplace = True)

etr2 = etr2.assign(sl3 =etr2[['101-150', '151-200','201-250']].mean(axis=1))
etr2['101-250'].fillna(value=etr2['sl3'], inplace=True)
etr2.drop(columns=['101-150', '151-200','201-250'],inplace=True)

etr2 = etr2.assign(sl4 =etr2[['201-300', '301-400','401-500']].mean(axis=1))
etr2['201-500'].fillna(value=etr2['sl4'], inplace=True)
etr2.drop(columns=['201-300', '301-400','401-500'],inplace=True)

etr2 = etr2.assign(sl5 =etr2[['601 & Above','801 & Above','901 & Above', '1201 & Above']].mean(axis=1))
etr2['601 & Above'].fillna(value=etr2['sl5'], inplace=True)
etr2.drop(columns=['801 & Above','901 & Above', '1201 & Above'],inplace=True)

etr2 = etr2.assign(sl6 =etr2[['151-250','151-300','151-400','301-500']].mean(axis=1))
etr2['201-500'].fillna(value=etr2['sl6'], inplace=True)
etr2.drop(columns=['151-250','151-300','151-400','301-500'],inplace=True)

etr2 = etr2.assign(sl7 =etr2[['0-120','0-150']].mean(axis=1))
etr2['0-200'].fillna(value=etr2['sl7'], inplace=True)
etr2.drop(columns=['0-120','0-150'],inplace=True)

etr2 = etr2.assign(sl8 =etr2[['101-250','121-240','201-500','251-500','401-600']].mean(axis=1))
etr2['201-600'] = etr2['sl8']
#etr2['201-600'].fillna(value=etr2['sl8'], inplace=True)
etr2.drop(columns=['101-250','121-240','201-500','251-500','401-600'],inplace=True)

etr2.drop(columns = ['sl3','sl4','sl5','sl6','sl7','sl8'],inplace=True)
etr2.drop(columns = ['101 & Above','401-800','601-900','801-1200'],inplace=True)

etr2 = etr2.assign(sl9 =etr2[['201 & Above','251 & Above', '301 & Above', '401 & Above', '501 & Above', '601 & Above']].mean(axis=1))
etr2['601 & Above'].fillna(value=etr2['sl9'], inplace=True)
etr2.drop(columns=['201 & Above','251 & Above','301 & Above','401 & Above','501 & Above','sl9'],inplace=True)

### changing column positions

In [ ]:
cols = etr2.columns.tolist()
cols.insert(2, cols.pop(cols.index('201-600')))
etr2 = etr2[cols]

In [ ]:
print(etr2.head())
list(etr2.columns)
#list(etr1.columns)

### Replacing missing values

In [ ]:
# Arunachal Pradesh
etr2.iloc[2,1] = 4 #http://power.arunachal.gov.in/wp-content/uploads/2018/06/2018-06-15-Tariff-Order-FY-2018-19-.pdf
etr2.iloc[2,2] = 4
etr2.iloc[2,3] = 4
#etr2.iloc[2,4] = 4

# Assam
etr2.iloc[3,0] = 4.65 # https://www.guwahatiplus.com/assam/apdcl-electricity-bills-to-increase-from-april-2022
etr2.iloc[3,1] = 6.6
etr2.iloc[3,2] = 7.6
etr2.iloc[3,3] = 7.6

#Bihar
etr2.iloc[4,1] = 6.1
etr2.iloc[4,2] = 6.4  #https://berc.co.in/orders/tariff/distribution/sbpdcl/2460-strong-span-style-color-000000-tariff-chart-for-fy-2022-23
etr2.iloc[4,3] = 6.7

#Jharkhand
etr2.iloc[15,2] = 6  #same for all slabs - https://jbvnl.co.in/Tariff/Tariff_jseb_2019.pdf
etr2.iloc[15,3] = 6

#Karnataka
etr2.iloc[16,2] = 7.9 #Tariff is applicable for 200 units and more

#Manipur
etr2.iloc[21,2] = 6.4 #Tariff is applicable for 200 units and more

#Meghalaya 
etr2.iloc[22,2] = 5.7 #Tariff is applicable for 200 units and more

#Mizoram
etr2.iloc[23,2] = 5.9 #Tariff is applicable for 200 units and more

etr2 = etr2.round(2)

### Saving to Excel file

In [ ]:
print(etr2)

In [ ]:
# Rename the columns
etr3 = etr2.rename(columns={'0-200': 'Lower', '201-600':'Middle','601 & Above': 'Higher'})
etr3 = etr3.replace('',np.nan)
etr3.to_excel('v2-el_price_rev.xlsx')
print(etr3)


In [ ]:
el_price = pd.read_excel('v2-el_price_rev.xlsx')
value = el_price.at[32,'Middle']
print(type(value))

In [ ]:
el_price = pd.read_excel('el_price_rev.xlsx')
print(el_price.columns)

In [ ]:
soc_eco_select = 'Middle'
state_1 = 'State'
el_price2 = el_price[['State',soc_eco_select]]
print(el_price2)

In [ ]:
state_name = 'Tripura'
el_price3 = el_price2.loc[el_price2['State']==state_name,soc_eco_select]
print(el_price3.item())